1\. Write a function that converts number representation (bin<->dec<->hex)

In [29]:
#function that converts a number from a decimal representation to a binary one
def dec_bin(n):
    n_bin = '0' + 'b'      #the binary string must begin with 0b 
    d = n
    s = n
    Mexp = 0
    while s >= 2:
        Mexp = Mexp + 1
        s = s/2    
    i = Mexp 
    while i >= 0:          #I simply use the expression of a decimal number in function of its binary representation, given in the tutorial 02            
        if d - 2**i >= 0 : 
            n_bin = n_bin + '1'
            d = d-2**i
        else:
            n_bin = n_bin + '0'
        i = i-1
    return n_bin


#function that converts a number from a decimal representation to a hexadecimal one
def dec_ex(n):
    n_ex = '0' + 'x'
    D = n
    S = n
    Mexp = 0
    while S >= 16:
        Mexp = Mexp + 1
        S = S//16
    i = Mexp 
        
    while i >= 0:             #I use the same template as above, but now there are more cases
        if D // 16**i == 15:
            n_ex = n_ex + 'f'
        elif D // 16**i == 14:
            n_ex = n_ex + 'e'
        elif D // 16**i == 13:
            n_ex = n_ex + 'd'
        elif D // 16**i == 12:
            n_ex = n_ex + 'c'
        elif D // 16**i == 11:
            n_ex = n_ex + 'b'
        elif D // 16**i == 10:
            n_ex = n_ex + 'a'
        else:
            n_ex = n_ex + str(D // 16**i)
        D = D - (D // 16**i)*(16**i)
        i = i - 1
    return n_ex


#function that converts one representation to another one. It uses the functions implemented above.
def convert(n):
    if(type(n) == int):
        dec_n = n   
        return dec_n, dec_bin(n), dec_ex(n)
    
    if (type(n) == str) and (n[1] == 'b'):
        bin_n = n
        bin_n_l = bin_n[2:]
        
        #from base 2 to base 10 
        dec_n = 0
        i = 1
        for c in bin_n_l:
            if c == '1':
                dec_n = dec_n + 2**(len(bin_n_l) - i)
            i = i + 1
        return dec_n, bin_n, dec_ex(dec_n)
    
    if  (type(n) == str) and (n[1] == 'x'):
            ex_n = n
            ex_n_l = ex_n[2:]
            
            #from hexadecimal to decimal
            dec_n = 0
            i = 1
            
            for c in ex_n_l:
                if c == '1':
                    dec_n = dec_n + 16**(len(ex_n_l) - i)
                elif c == '2':
                     dec_n = dec_n + 2*16**(len(ex_n_l) - i)
                elif c == '3':
                    dec_n = dec_n + 3*16**(len(ex_n_l) - i)
                elif c == '4':
                    dec_n = dec_n + 4*16**(len(ex_n_l) - i)
                elif c == '5':
                    dec_n = dec_n + 5*16**(len(ex_n_l) - i)
                elif c == '6':
                    dec_n = dec_n + 6*16**(len(ex_n_l) - i)
                elif c == '7':
                    dec_n = dec_n + 7*16**(len(ex_n_l) - i)
                elif c == '8':
                    dec_n = dec_n + 8*16**(len(ex_n_l) - i)
                elif c == '9':
                    dec_n = dec_n + 9*16**(len(ex_n_l) - i) 
                elif c == 'a':
                    dec_n = dec_n + 10*16**(len(ex_n_l) - i)
                elif c == 'b':
                    dec_n = dec_n + 11*16**(len(ex_n_l) - i)
                elif c == 'c':
                    dec_n = dec_n + 12*16**(len(ex_n_l) - i)
                elif c == 'd':
                    dec_n = dec_n + 13*16**(len(ex_n_l) - i)
                elif c == 'e':
                    dec_n = dec_n + 14*16**(len(ex_n_l) - i)
                elif c == 'f':
                    dec_n = dec_n + 15*16**(len(ex_n_l) - i)
                i = i + 1
            
            return dec_n, dec_bin(dec_n), ex_n

#let's test the convert function
for i in range(0,1000):
    if hex(i) != convert(i)[2]: print('-1')
    elif bin(i) != convert(i)[1]: print('-2')
    elif hex(i) != convert(bin(i))[2]: print('-3')
    elif i != convert(bin(i))[0]: print('-4')
    elif i != convert(hex(i))[0]: print('-5')
    elif bin(i) != convert(hex(i))[1]: print('-6')

 

    
   

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [30]:
def spn_num(x):
    if len(x) != 32: return -1 #error message if the argument given to spn_num is wrong
    else:
        sgn = 0
        exp = '0' + 'b'
        mnt = 0
        if x[0] == '1':
            sgn = -1
        else:
            sgn = 1
        exp = exp + x[1:9]
        i = 1
        for c in x[9:]:
            if c == '1':
                mnt = mnt + 2**(-i)
            i = i + 1
        exp = convert(exp)[0]
        return sgn*(1 + mnt)*2**(exp - 127)

#let's test spn_num function, examples prova and prova2 are the ones given in the tutorial 02
prova = ''
i = 0
while i < 32:
    if i == 6 or i == 7 or i == 8 or i == 9 or i == 10:
        prova = prova + '1'
    else:    
        prova = prova + '0'
    i = i + 1

prova2 = ''
i = 0
while i < 32:
    if i == 0 or i == 1 or i == 8 or i == 10 or i == 11:
        prova2 = prova2 + '1'
    else:    
        prova2 = prova2 + '0'
    i = i + 1

prova3 = ''
i = 0
while i < 32:
    if i == 31:
        prova3 = prova3 + '1'
    else:    
        prova3 = prova3 + '0'
    i = i + 1
    
print('binary code:', prova, 'number:',spn_num(prova))   
print('binary code:', prova2, 'number:', spn_num(prova2))
print('binary code:', prova3, 'number:', spn_num(prova3))



binary code: 00000011111000000000000000000000 number: 1.316553672920962e-36
binary code: 11000000101100000000000000000000 number: -5.5
binary code: 00000000000000000000000000000001 number: 5.87747245476067e-39


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [31]:
ufl = 0.25
ofl = 1

i = 0
while len(str(ufl)) > 3: #when python reaches the uderflow-limit it starts to print 0.0
    ufl = ufl / 2
    i = i + 1

ufl = 0.25*0.5**(i-1)
print('starting point', 0.25, '\nnumber of iterations:', i, '\nvalue before the underflow limit:',ufl)

#I cannot find the over-flow limit
'''  
i = 0
while i < 600000:
    ofl = ofl*2
    i = i + 1
    if i > 599998:
        print(i,ofl)
        print('\n\n')
'''

starting point 0.25 
number of iterations: 1073 
value before the underflow limit: 5e-324


"  \ni = 0\nwhile i < 600000:\n    ofl = ofl*2\n    i = i + 1\n    if i > 599998:\n        print(i,ofl)\n        print('\n\n')\n"

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [32]:
a = 1
b = 0.24

counter = 0
while (len(str(a)) > 4) or (a == 1.12) or (a == 1) :
    counter += 1 
    b = b / 2
    a = a + b

print('number of iterations:',counter)
print('min value of addendum:', b)
print('value before the limit:', a - b, '\nlimit:', a, '\nvalue after the limit:',a + b/2)


number of iterations: 50
min value of addendum: 2.1316282072803005e-16
value before the limit: 1.2399999999999998 
limit: 1.24 
value after the limit: 1.24


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [33]:
import numpy as np
import math

def root(a,b,c):
    return (-b + math.sqrt(b**2 - 4*a*c))/(2*a), (-b - math.sqrt(b**2 - 4*a*c))/(2*a)

def root2(a,b,c):
    return (2*c)/(-b - math.sqrt(b**2 - 4*a*c)), (2*c)/(-b + math.sqrt(b**2 - 4*a*c))

#python built-in function
coeff = [1e-3,1e3,1e-3]
C = np.roots(coeff)

print('SOLUTIONS:')
print('python built in function:', '(', C[1], C[0], ')')
print('root version 1:', root(1e-3, b = 1e3, c = 1e-3))
print('root version 2:', root2(1e-3, b = 1e3, c = 1e-3))

#---------------------------Comment---------------------------------------------------------------
# The solutions are similar. 
#Supposing that the Python built-in function shows the best roots,
#I notice that 'root' version 1 returns in a correct way the major solution, and 'root' version 2 the minor one.  
#The issue is the sum between the coefficient '-b' and the square root of the discriminant.
#The best function I can write is maybe a mix of the two above.
#-----------------------------------------------------------------------------------------------------

def root_mix(a,b,c):
    return (2*c)/(-b - math.sqrt(b**2 - 4*a*c)), (-b - math.sqrt(b**2 - 4*a*c))/(2*a)

print('root version 3:', root_mix(1e-3, b = 1e3, c = 1e-3))

SOLUTIONS:
python built in function: ( -1.000000000001e-06 -999999.999999 )
root version 1: (-9.999894245993346e-07, -999999.999999)
root version 2: (-1.000000000001e-06, -1000010.5755125057)
root version 3: (-1.000000000001e-06, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [34]:
def f(x):
    return x*(x-1)

#the right result of the derivative is 1
i = 0
while i<= 12:
    d = 10**(-2-i)
    l = (f(1+d)-f(1))/d
    print('limit:' ,l, 'delta:', d)
    i = i + 2

#the accuracy improves until delta = 1e-8 and then get worse.  

limit: 1.010000000000001 delta: 0.01
limit: 1.0000999999998899 delta: 0.0001
limit: 1.0000009999177333 delta: 1e-06
limit: 1.0000000039225287 delta: 1e-08
limit: 1.000000082840371 delta: 1e-10
limit: 1.0000889005833413 delta: 1e-12
limit: 0.9992007221626509 delta: 1e-14


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [35]:
import time
from math import pi
'''
import numpy as np
import time 
k = np.arange(1,10000001)
N = 10000000
#print(k)
I = (2/N)*sum(1 - (k/N)**2)
print(I)#già la prima cifra decimale è sbagliata
'''
def I(N):
    N = int(N)
    I = 0.0
    for k in range (1,N+1):
        I = I + (2/N)*(1 - (k/N)**2)
    return I

print('------------------------a----------------------------------')
print('value of the integral with 100 iterations:',I(100) )
print('percentage error:', (((pi/2)-I(100))/(pi/2))*100,'%\n')


start_time = time.time()
i = 111*1e4
while (time.time() - start_time) <= 1:
    start_time = time.time()
    g = I(i)
    i = i + 1e3

print('------------------------b---------------------------------------')
print('number of iterations:', i-1e3 , '\nvalue of the integral:', g , '\ntime needed:', time.time() - start_time)
print('percentage error:', (((pi/2)-g)/(pi/2))*100,'%\n')

start_time = time.time()
h = I(7 * 1e7)

print('------------------------------------------------------------------')
print('number of iterations:', 7*1e7 , '\nvalue of the integral:', h , '\ntime needed:', time.time() - start_time)
print('percentage error:', (((pi/2)-h)/(pi/2))*100,'%\n')

#increasing the number of iterations the value of the integral get better very slowly. 

------------------------a----------------------------------
value of the integral with 100 iterations: 1.3233000000000008
percentage error: 15.756105522597908 %

------------------------b---------------------------------------
number of iterations: 1110000.0 
value of the integral: 1.3333324324321278 
time needed: 1.0011882781982422
percentage error: 15.117421037474521 %

------------------------------------------------------------------
number of iterations: 70000000.0 
value of the integral: 1.333333319048169 
time needed: 63.392245054244995
percentage error: 15.117364593744284 %

